### Initialization

In [1]:
import numpy as np
import pandas as pd
from sys import stderr
from scipy.stats import rankdata

In [2]:
def percentile(matrix, p):
    """
    Estimation of percentile without zeros
    
    Parameters
    ----------
    matrix : array_like
        Matrix to calculate percentile.
    p : float in range of [0,100]
        Percentile to compute, must be between 0 and 100 inclusive.
        
    Returns
    -------
    float
        Сalculated percentile.
    """
    return np.percentile(matrix[np.any(matrix > 0, axis=1)], p, axis=0)

In [3]:
def tmm_normalization(matr, index_ref=None, trim_fold_change=0.3, trim_abs_expr=0.05):
    """
    Trimmed mean of M-values normalization
    
    Parameters
    ----------
    matrix : array_like
        Matrix to normalize.
    index_ref:
        Index of reference column.
    trim_fold_change:
        Percent of trimmed for folder change.
    trim_abs_expr:
        Percent of trimmed for absolute expression.
        
    Returns
    -------
    array_like
        Normalized matrix.
    """
    matrix = np.array(matr)                           # better speed of calculating
    np.seterr(divide='ignore', invalid='ignore')      # for divide on zeros in log2
    
    # Calculation log2(tmm_factor)
    def log2_tmm(index_vec):
        # select the necessary vectors
        curr_vec = matrix[:, index_vec]
        ref_vec = matrix[:, index_ref]
        
        # total number molecules in cells
        total_curr_vec = np.sum(curr_vec)
        total_ref_vec = np.sum(ref_vec)
        
        # select significant genes
        check_inf = (~np.isinf(matr_a[:, index_vec])) & (~np.isinf(matr_m[:, index_vec]))
        ranks = rankdata(matr_a[:, index_vec][check_inf], method='ordinal')
        bool_a = (ranks > len(ranks) * trim_abs_expr) & (ranks < len(ranks) * (1 - trim_abs_expr))
        ranks = rankdata(matr_m[:, index_vec][check_inf], method='ordinal')
        bool_m = (ranks > len(ranks) * trim_fold_change) & (ranks < len(ranks) * (1 - trim_fold_change))
        curr_vec = curr_vec[check_inf]
        ref_vec = ref_vec[check_inf]
        bool_curr_vec = curr_vec > 0
        bool_ref = ref_vec > 0
        bool_result = bool_curr_vec & bool_ref & bool_a & bool_m
        
        # сalculation of required values
        w_vec = 1 / ((total_curr_vec - curr_vec[bool_result]) / (total_curr_vec * curr_vec[bool_result]) + 
                     (total_ref_vec - ref_vec[bool_result]) / (total_ref_vec * ref_vec[bool_result]))
        m_vec = np.log2(curr_vec[bool_result] / total_curr_vec) - np.log2(ref_vec[bool_result] / total_ref_vec)
        
        # calculation log2(tmm_factor)
        w_sum = np.sum(w_vec)
        if np.isclose(w_sum, 0) or np.isinf(w_sum):
            print("Unexpected sum of weights for vector {}: '{}'".format(index_vec, w_sum), file=stderr)
            return 0
        
        return np.sum(w_vec * m_vec) / w_sum
        
    # find index of reference column
    f75 = percentile(matrix, 75)
    if index_ref is None:
        index_ref = np.argmin(abs(f75 - np.mean(f75)))
    elif isinstance(numeric_matrix, pd.DataFrame) and isinstance(index_ref, int):
        index_ref = matr.columns.values[index_ref]    
    
    # find matrix A and M described expression levels of genes
    matr_norm = matrix / np.sum(matrix, axis=0)
    matr_a = np.log2(matr_norm * matr_norm[:, index_ref].reshape(matr_norm.shape[0], 1)) / 2
    matr_m = np.log2(matr_norm / matr_norm[:, index_ref].reshape(matr_norm.shape[0], 1))
    
    # calculation tmm_factor and normalization of input data
    tmm_factor = 2 ** np.array([log2_tmm(i) for i in range(matrix.shape[1])])
    return matr / tmm_factor

### Numeric test

In [4]:
numeric_matrix = pd.read_table('../../test_data/numeric_dataset_1.txt', sep=' ', header=None)

Before:

In [5]:
numeric_matrix

,0,1,2
0,5,10,1
1,10,20,2
2,1,2,5
3,353,0,2
4,7,7,7
5,8,9,8


After:

In [6]:
normal_numeric_matrix = tmm_normalization(numeric_matrix)
normal_numeric_matrix

,0,1,2
0,5.0,0.865236,0.122457
1,10.0,1.730471,0.244914
2,1.0,0.173047,0.612285
3,353.0,0.000000,0.244914
4,7.0,0.605665,0.857199
5,8.0,0.778712,0.979656


### Perfomance test

In [7]:
matrix = np.random.randint(0, 1000, size=(3000, 5000))

print("TrimmedMeanNormalization - ", end="")
%time normal_matrix = tmm_normalization(matrix)

TrimmedMeanNormalization - Wall time: 8.32 s


Read matrix from .csv file and normalize it:

In [8]:
data = pd.read_csv("../../test_data/srr1784310_subset.csv", index_col=0)
data.head()

,AGCACCTCTAAGCTTCT,GAGACAGATACGCTAGTC,CCAACCGTCGATTGAT,TGATATTGCCTAACAATCC,ATATGCATACTAGGAT,GACTAGACCCAAACGCCT,ACCTTGCCAAACCTCC,GATGACCCTCACCTTGCC,AATATACCTATATGCAT,GATAACCATCCCTCGTCT,...,TGAATGCATGGGGTTAGTG,TGAAGCGTAGGGAACGATT,CCCATCTGTTATCTGT,ATCATGAGGTAGTCTAG,TGATACGTGCTTGACGGAC,AGGTCACAGGCATGGGT,TGAGTTCTGTTGGGAACCT,GAGGTCCCTTCGACTCCT,GATTAGACCGGCTTAC,GTTCAACTGGTTAGTG
uc009vfc.1,49,50,48,33,47,42,40,21,171,28,...,4,1,3,5,8,5,2,3,4,12
uc009vew.1,50,45,57,28,72,59,44,40,155,58,...,10,2,4,3,1,4,4,6,5,10
Lars2,90,110,89,113,77,81,57,91,62,71,...,11,14,15,10,10,17,18,20,5,24
Hsp90ab1,143,103,104,112,111,90,94,125,55,84,...,12,13,11,6,15,3,14,9,10,11
Ptma,67,136,85,66,96,73,74,80,34,64,...,11,10,5,3,10,7,14,7,16,8


In [9]:
normal_data = tmm_normalization(data)
normal_data.head()

,AGCACCTCTAAGCTTCT,GAGACAGATACGCTAGTC,CCAACCGTCGATTGAT,TGATATTGCCTAACAATCC,ATATGCATACTAGGAT,GACTAGACCCAAACGCCT,ACCTTGCCAAACCTCC,GATGACCCTCACCTTGCC,AATATACCTATATGCAT,GATAACCATCCCTCGTCT,...,TGAATGCATGGGGTTAGTG,TGAAGCGTAGGGAACGATT,CCCATCTGTTATCTGT,ATCATGAGGTAGTCTAG,TGATACGTGCTTGACGGAC,AGGTCACAGGCATGGGT,TGAGTTCTGTTGGGAACCT,GAGGTCCCTTCGACTCCT,GATTAGACCGGCTTAC,GTTCAACTGGTTAGTG
uc009vfc.1,53.301170,53.608418,49.644894,33.632394,50.309627,41.099664,41.225593,21.982511,166.652106,29.660589,...,2.705592,0.683386,1.904134,2.689195,5.464330,3.519318,1.347474,2.023196,2.819278,7.754704
uc009vew.1,54.388949,48.247576,58.953312,28.536576,77.070067,57.735242,45.348153,41.871450,151.058926,61.439791,...,6.763980,1.366773,2.538845,1.613517,0.683041,2.815454,2.694948,4.046392,3.524097,6.462254
Lars2,97.900109,117.938520,92.049908,115.165469,82.422155,79.263638,58.746471,95.257549,60.423571,75.210778,...,7.440378,9.567408,9.520669,5.378390,6.830413,11.965680,12.127265,13.487974,3.524097,15.509408
Hsp90ab1,155.552395,110.433341,107.563938,114.146305,118.816354,88.070709,96.880145,130.848282,53.601554,88.981766,...,8.116776,8.884022,6.981824,3.227034,10.245619,2.111591,9.432318,6.069588,7.048194,7.108479
Ptma,72.881192,145.814897,87.912834,67.264787,102.760090,71.435130,76.267348,83.742901,33.135506,67.795631,...,7.440378,6.833863,3.173556,1.613517,6.830413,4.927045,9.432318,4.720791,11.277111,5.169803
